In [42]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BASE_DIR = r"D:/Practicle/DL/dataset/Object Detection(Ass6)"
DATASET_DIR = BASE_DIR + "/caltech-101-img"
WEIGHTS_PATH = BASE_DIR + "/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


In [43]:
base_model = VGG16(
    weights=WEIGHTS_PATH,
    include_top=False,
    input_shape=(224, 224, 3)
)


In [44]:
for layer in base_model.layers:
    layer.trainable = False


In [45]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(102, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [46]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 7357 images belonging to 102 classes.
Found 1788 images belonging to 102 classes.


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Epoch 1/10
 45/230 ━━━━━━━━━━━━━━━━━━━━ 6:53 2s/step - accuracy: 0.1123 - loss: 4.3073

In [ ]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_ft = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


In [ ]:
model.save(BASE_DIR + "/vgg16_finetuned_caltech101.h5")
print("Model saved successfully!")
